<a href="https://colab.research.google.com/github/KIMMOOKYONG/COLAB-NOTENOOK/blob/main/sqlite%EC%82%AC%EC%9A%A9%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.22.0


# 데이터베이스 연결

## 메모리 DB 접속 (일회성)

In [38]:
import sqlite3
con = sqlite3.connect(':memory:')

## 파일 DB 접속

In [ ]:
# # DB 생성 (오토 커밋)
# conn = sqlite3.connect("test.db", isolation_level=None)

# # 커서 획득
# c = conn.cursor()

# # 테이블 생성 (데이터 타입은 TEST, NUMERIC, INTEGER, REAL, BLOB 등)
# c.execute("CREATE TABLE IF NOT EXISTS table1 \
#     (id integer PRIMARY KEY, name text, birthday text)")

In [32]:
# DB 생성 (오토 커밋)
conn = sqlite3.connect("test.db", isolation_level=None)

# 커서 획득
c = conn.cursor()

# 데이터 삽입하기

## 기본 String Query 사용

In [33]:
# 데이터 삽입 방법 1
c.execute("INSERT INTO table1 \
    VALUES(1, 'LEE', '1987-00-00')")

## Parameter: Tuple 사용

In [34]:
# 데이터 삽입 방법 2
c.execute("INSERT INTO table1(id, name, birthday) \
    VALUES(?,?,?)", \
    (2, 'KIM', '1990-00-00'))

In [35]:
test_tuple = (
    (3, 'PARK', '1991-00-00'),
    (4, 'CHOI', '1999-00-00'),
    (5, 'JUNG', '1989-00-00')
)

In [36]:
c.executemany("INSERT INTO table1(id, name, birthday) VALUES(?,?,?)", test_tuple)

## Named Parameter: Dictionary 사용

In [ ]:
name = 'SangJung'
phoneNumber = '010-5670-2343'
cur = con.cursor()
cur.execute('INSERT INTO PhoneBook VALUES(:name, :phoneNumber);', {"name":name, "phoneNumber":phoneNumber})

## List 사용

In [ ]:
dataList = (('Tom', '010-543-5432'), ('DSP', '010-123-1234'))
cur = con.cursor()
cur.executemany("INSERT INTO PhoneBook VALUES(?, ?);", dataList)

# 데이터 불러오기

## 순회 조회

In [ ]:
cur.execute('SELECT * FROM PhoneBook')
for row in cur:
    print(row)

## 단건 조회

In [19]:
c.execute('select * from table1')
print(c.fetchone())

(1, 'LEE', '1987-00-00')


In [18]:
c.execute('select * from table1')
print(c.fetchall())

[(1, 'LEE', '1987-00-00'), (2, 'KIM', '1990-00-00'), (3, 'PARK', '1991-00-00'), (4, 'CHOI', '1999-00-00'), (5, 'JUNG', '1989-00-00')]


## 다건 조회

In [ ]:
cur.fetchmany(2)

## 모두 조회

In [20]:
# 방법 1
c.execute("SELECT * FROM table1")
for row in c.fetchall():
    print(row)

(1, 'LEE', '1987-00-00')
(2, 'KIM', '1990-00-00')
(3, 'PARK', '1991-00-00')
(4, 'CHOI', '1999-00-00')
(5, 'JUNG', '1989-00-00')


In [21]:
# 방법 2
for row in c.execute("SELECT * FROM table1 ORDER BY id ASC"):
    print(row)

(1, 'LEE', '1987-00-00')
(2, 'KIM', '1990-00-00')
(3, 'PARK', '1991-00-00')
(4, 'CHOI', '1999-00-00')
(5, 'JUNG', '1989-00-00')


# 데이터 조회하기 (필터링)

In [22]:
# 방법 1
param1 = (1,)
c.execute('SELECT * FROM table1 WHERE id=?', param1)
print('param1', c.fetchone())
print('param1', c.fetchall())

param1 (1, 'LEE', '1987-00-00')
param1 []


In [23]:
# 방법 2
param2 = 1
c.execute("SELECT * FROM table1 WHERE id='%s'" % param2)  # %s %d %f
print('param2', c.fetchone())
print('param2', c.fetchall())

param2 (1, 'LEE', '1987-00-00')
param2 []


In [24]:
# 방법 3
c.execute("SELECT * FROM table1 WHERE id=:Id", {"Id": 1})
print('param3', c.fetchone())
print('param3', c.fetchall())

param3 (1, 'LEE', '1987-00-00')
param3 []


In [25]:
# 방법 4
param4 = (1, 4)
c.execute('SELECT * FROM table1 WHERE id IN(?,?)', param4)
print('param4', c.fetchall())

param4 [(1, 'LEE', '1987-00-00'), (4, 'CHOI', '1999-00-00')]


In [26]:
# 방법 5
c.execute("SELECT * FROM table1 WHERE id In('%d','%d')" % (1, 4))
print('param5', c.fetchall())

param5 [(1, 'LEE', '1987-00-00'), (4, 'CHOI', '1999-00-00')]


In [27]:

# 방법 6
c.execute("SELECT * FROM table1 WHERE id=:id1 OR id=:id2", {"id1": 1, "id2": 4})
print('param6', c.fetchall())

param6 [(1, 'LEE', '1987-00-00'), (4, 'CHOI', '1999-00-00')]


# 데이터 수정하기

In [28]:
# 방법 1
c.execute("UPDATE table1 SET name=? WHERE id=?", ('NEW1', 1))
# 방법 2
c.execute("UPDATE table1 SET name=:name WHERE id=:id", {"name": 'NEW2', 'id': 3})
# 방법 3
c.execute("UPDATE table1 SET name='%s' WHERE id='%s'" % ('NEW3', 5))
# 확인
for row in c.execute('SELECT * FROM table1'):
    print(row)

(1, 'NEW1', '1987-00-00')
(2, 'KIM', '1990-00-00')
(3, 'NEW2', '1991-00-00')
(4, 'CHOI', '1999-00-00')
(5, 'NEW3', '1989-00-00')


# 데이터 삭제하기

In [29]:
# 방법 1
conn.execute("DELETE FROM table1")

In [30]:
# 방법 2
print(conn.execute("DELETE FROM table1").rowcount)

0


# DB 연결 해제

In [31]:
conn.close()

# DB 백업하기 (dump)

In [37]:
with conn:
    with open('dump.sql', 'w') as f:
        for line in conn.iterdump():
            f.write('%s\n' % line)
        print('Completed.')

Completed.


# sqlite와 pandas 연동하기

In [1]:
import sqlite3

In [15]:
conn = sqlite3.connect("employee.db")
cur = conn.cursor()

In [18]:
conn.execute("create table if not exists employee_data(\
id integer, name text, nickname text, department text,\
employee_date text)")

conn.executemany(
    "insert into employee_data values(?,?,?,?,?)",
    [
        (1001, 'Donghyun', 'SOMJANG', 'Development', '2020-04-01 00:00:00.000'),
        (2001, 'Sol', 'Fairy', 'Marketing', '2020-04-01 00:00:00.000'),
        (2002, 'Jiyoung', 'Magician', 'Marketing', '2020-04-01 00:00:00.000'),
        (1002, 'Hyeona', 'Theif', 'Development', '2020-04-01 00:00:00.000'),
        (1003, 'Soyoung', 'Chief', 'Development', '2020-04-01 00:00:00.000'),
    ]
)

conn.commit()
conn.close()

In [19]:
conn = sqlite3.connect("employee.db")
cur = conn.cursor()
cur.execute("select * from employee_data")
rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()   

(1001, 'Donghyun', 'SOMJANG', 'Development', '2020-04-01 00:00:00.000')
(1001, 'Donghyun', 'SOMJANG', 'Development', '2020-04-01 00:00:00.000')
(2001, 'Sol', 'Fairy', 'Marketing', '2020-04-01 00:00:00.000')
(2002, 'Jiyoung', 'Magician', 'Marketing', '2020-04-01 00:00:00.000')
(1002, 'Hyeona', 'Theif', 'Development', '2020-04-01 00:00:00.000')
(1003, 'Soyoung', 'Chief', 'Development', '2020-04-01 00:00:00.000')


In [22]:
conn = sqlite3.connect("employee.db")
cur = conn.cursor()
cur.execute("select name, department from employee_data where employee_data.id > 2000")

rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()  

('Sol', 'Marketing')
('Jiyoung', 'Marketing')


In [24]:
conn = sqlite3.connect("employee.db")
cur = conn.cursor()
cur.execute("select nickname from employee_data where employee_data.name == 'Donghyun'")

rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()  

('SOMJANG',)
('SOMJANG',)


In [25]:
import pandas as pd
conn = sqlite3.connect("employee.db")
cur = conn.cursor()
cur.execute("select * from employee_data")
rows = cur.fetchall()

cols = [column[0] for column in cur.description]
data_df = pd.DataFrame.from_records(data=rows, columns=cols)
conn.close()

data_df

,id,name,nickname,department,employee_date
0,1001,Donghyun,SOMJANG,Development,2020-04-01 00:00:00.000
1,1001,Donghyun,SOMJANG,Development,2020-04-01 00:00:00.000
2,2001,Sol,Fairy,Marketing,2020-04-01 00:00:00.000
3,2002,Jiyoung,Magician,Marketing,2020-04-01 00:00:00.000
4,1002,Hyeona,Theif,Development,2020-04-01 00:00:00.000
5,1003,Soyoung,Chief,Development,2020-04-01 00:00:00.000
